## Please Download the gguf file (model_path)

download this: Llama-3.2-1B-Instruct-IQ3_M.gguf
link: https://huggingface.co/bartowski/Llama-3.2-1B-Instruct-GGUF/blob/main/Llama-3.2-1B-Instruct-IQ3_M.gguf

Install some dependencies

In [ ]:
!pip install -q -U accelerate==0.27.1
!pip install -q -U datasets==2.17.0
!pip install -q -U transformers==4.38.1
!pip install langchain sentence-transformers chromadb langchainhub

!pip install langchain-community langchain-core

Get the Model You Want

In [ ]:
#!pip install --upgrade --upgrade-strategy eager "optimum[neural-compressor]"
!pip install llama-cpp-python

In [17]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
import chromadb

import time
import uuid
from llama_cpp import Llama
#from fuzzywuzzy import fuzz
#from optimum.quantization import QuantizeruenceClassification


Define Variables

In [23]:
from llama_cpp import Llama

model_path = "/content/Llama-3.2-1B-Instruct-IQ3_M.gguf"

model = Llama(model_path=model_path, n_ctx=2048, n_threads=8)


llama_model_loader: loaded meta data with 35 key-value pairs and 147 tensors from /content/Llama-3.2-1B-Instruct-IQ3_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Llama 3.2 1B Instruct
llama_model_loader: - kv   3:                           general.finetune str              = Instruct
llama_model_loader: - kv   4:                           general.basename str              = Llama-3.2
llama_model_loader: - kv   5:                         general.size_label str              = 1B
llama_model_loader: - kv   6:                            general.license str              = llama3.2
llama_model_loader: - kv   7:             

In [19]:
ls -lh /content/Llama-3.2-1B-Instruct-IQ3_M.gguf


-rw-r--r-- 1 root root 37M Apr  3 03:34 /content/Llama-3.2-1B-Instruct-IQ3_M.gguf


Define Data Sources

In [24]:
import pandas as pd

file_names = [
    "study_permit_general",
    "work_permit_student_general",
    "work-study-data-llm",
    "vancouver_transit_qa_pairs",
    "permanent_residence_student_general",
    "data-with-sources"
]

all_texts = []

for file in file_names:
    path = f'./sample_data/{file}.csv'
    try:
        df = pd.read_csv(path)
        df.columns = df.columns.str.lower()

        if 'question' in df.columns and 'answer' in df.columns:
            df = df.drop_duplicates(subset=['question'])
            df['text'] = df['question'].fillna('') + ' ' + df['answer'].fillna('')
        else:
            print(f"no text columns in {file}")
            continue
        all_texts.extend(df['text'].tolist())
    except Exception as e:
        print(f"Error loading {file}: {e}")


Set Embedding Model, and Chroma Client to Interact w Vector Database and Create Collections

In [25]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
import chromadb
import uuid
import pandas as pd
#from fuzzywuzzy import fuzz

# pt model for generating embeddings used pretty often
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

# persistent client to interact w chroma vector store
client = chromadb.PersistentClient(path="./chroma_db")

# create collections for each data (for testing rn)
collection = client.get_or_create_collection(name="combined_docs")

# seems like better results if we remove duplicates and very similar data
data = pd.DataFrame({"text": all_texts})
data = data.drop_duplicates()
all_texts = data["text"].tolist()

print(f"successfully added {len(all_texts)} documents to Chroma DB.")

successfully added 1031 documents to Chroma DB.


Function to add data to collection by embedding them

In [26]:
def add_data_to_collection_batch(collection, texts, batch_size=3):
    for idx in range(0, len(texts), batch_size):
        try:
            batch_texts = texts[idx: idx + batch_size]

            embeddings = embedding_model.embed_documents(batch_texts)

            batch_ids = [str(uuid.uuid4()) for _ in batch_texts]

            collection.add(
                ids=batch_ids,
                embeddings=embeddings,
                documents=batch_texts
            )
            print(f"successfully added {len(batch_texts)} documents (Batch {idx}-{idx + batch_size - 1})")
        except Exception as e:
            print(f"Error processing batch starting at index {idx}: {e}")

In [ ]:
add_data_to_collection_batch(collection, all_texts)
print(f"successfully added {len(all_texts)} documents to the Chroma collection.")

Function to now match for releveant document

In [28]:
def get_relevant_documents(query, n_results=3):
    try:
        query_embeddings = embedding_model.embed_documents([query])[0]

        results = collection.query(query_embeddings=[query_embeddings], n_results=n_results)
        print(f"Query Results: {results}")

        return results['documents'][0] if results['documents'] else []
    except Exception as e:
        print(f"Error querying: {e}")
        return []

Generate Answer

In [29]:
def generate_answer(query):
    response_before_rag = model(query, max_tokens=200, temperature=0.1)["choices"][0]["text"]

    relevant_documents = get_relevant_documents(query)
    if not relevant_documents:
        return {
            "Before RAG Response": response_before_rag,
            "After RAG Response": "Sorry, no relevant documents found."
        }

    relevant_texts = "\n\n".join(relevant_documents)
    rag_prompt = f"""
    You are a helpful assistant for international students. Here are relevant documents:

    {relevant_texts}

    Please respond to the following question based on the documents above. Be conversational but concise:

    Question: {query}

    Answer:
    """

    response_after_rag = model(rag_prompt, max_tokens=300, temperature=0.1)["choices"][0]["text"]

    return {
        "Before RAG Response": response_before_rag,
        "After RAG Response": response_after_rag
    }


Example Usage

In [30]:
test_queries = [
    "How do I apply for a study permit in Canada?",
    "Can I work while studying on a student visa?",
    "What happens if my study permit expires before I finish my program?",
    "Do I need a new study permit if I change schools?",
    "How long does it take to process a Canadian study permit?",
    "Am I allowed to work off-campus as an international student?",
    "How many hours can I work while studying in Canada?",
    "What documents do I need to apply for a co-op work permit?",
    "Can I work in Canada after I graduate?",
    "What is a Post-Graduation Work Permit (PGWP) and how do I apply?",
    "How do I apply for MSP (Medical Services Plan) in British Columbia?",
    "Is MSP mandatory for international students?",
    "What healthcare services are covered under MSP?",
    "What should I do if I get sick and don’t have insurance yet?",
    "Can I use private health insurance instead of MSP?",
    "What are my options for student housing in Vancouver?",
    "How much does rent typically cost for international students?",
    "What should I check before signing a lease in Canada?",
    "Are there any student discounts for accommodation?",
    "How can I find a roommate in Canada?",
    "How do I open a bank account as an international student?",
    "What documents do I need to get a student bank account?",
    "Can I get a credit card as an international student?",
    "How do I send money to my home country from Canada?",
    "What scholarships are available for international students?",
    "How does the Compass Card work for transit in Vancouver?",
    "Am I eligible for a U-Pass as an international student?",
    "What is the best way to get around Vancouver on a budget?",
    "Where can I find the bus and SkyTrain schedules?",
    "Are there student discounts for public transportation?",
    "Can I apply for permanent residence after graduating?",
    "What is the Canadian Experience Class (CEC) immigration program?",
    "How can I improve my chances of getting permanent residence?",
    "What are the eligibility requirements for Express Entry?",
    "Does having a Canadian degree help with PR applications?"
]

for idx, user_query in enumerate(test_queries, start=1):
    responses = generate_answer(user_query)

    print("\n" + "="*50)
    print(f"Test Query {idx}: {user_query}")
    print("="*50)
    print("Response Before RAG:\n", responses["Before RAG Response"])
    print("\nResponse After RAG:\n", responses["After RAG Response"])
    print("="*50 + "\n\n")

#add_all_texts_to_collection()

# big speedup


llama_perf_context_print:        load time =    1449.26 ms
llama_perf_context_print: prompt eval time =    1448.96 ms /    12 tokens (  120.75 ms per token,     8.28 tokens per second)
llama_perf_context_print:        eval time =   32282.04 ms /   199 runs   (  162.22 ms per token,     6.16 tokens per second)
llama_perf_context_print:       total time =   33973.87 ms /   211 tokens
Llama.generate: 1 prefix-match hit, remaining 556 prompt tokens to eval


Query Results: {'ids': [['18a103b4-593d-4ad1-93db-9c9249abc800', '8a934d09-1d19-4add-86e3-41949ff57026', 'cc8455ae-1572-4740-94c0-e522edcd67c8']], 'embeddings': None, 'documents': [['How do I pay for a study permit?("https://ircc.canada.ca/english/helpcentre/answer.asp?qnum=481&top=15") You must pay a processing fee with your application. For details, visit the\xa0Pay your fees page("https://ircc.canada.ca/english/information/fees/index.asp")\xa0to learn about the methods of payment accepted by Canadian visa offices.', 'How do I pay for a study permit?("https://ircc.canada.ca/english/helpcentre/answer.asp?qnum=481&top=15") You must pay a processing fee with your application. For details, visit the\xa0Pay your fees page("https://ircc.canada.ca/english/information/fees/index.asp")\xa0to learn about the methods of payment accepted by Canadian visa offices.', 'Do I need a permit to study in Canada?("https://ircc.canada.ca/english/helpcentre/answer.asp?qnum=478&top=15") Most\xa0foreign nati

llama_perf_context_print:        load time =    1449.26 ms
llama_perf_context_print: prompt eval time =   67690.51 ms /   556 tokens (  121.75 ms per token,     8.21 tokens per second)
llama_perf_context_print:        eval time =   49397.92 ms /   299 runs   (  165.21 ms per token,     6.05 tokens per second)
llama_perf_context_print:       total time =  117477.65 ms /   855 tokens
Llama.generate: 1 prefix-match hit, remaining 10 prompt tokens to eval



Test Query 1: How do I apply for a study permit in Canada?
Response Before RAG:
  The process typically involves the following steps:

1. **Determine your eligibility**: Check if you meet the requirements for a study permit, which typically includes having a valid passport, proof of sufficient funds, and meeting the language proficiency requirements.
2. **Gather required documents**: Collect the necessary documents, such as:
	* A valid passport
	* Proof of sufficient funds (e.g., bank statements, income verification)
	* Proof of language proficiency (e.g., language test results, language test score)
	* Proof of acceptance to a Canadian post-secondary institution (if applicable)
	* Police certificate (if required)
3. **Submit your application**: Fill out the study permit application form (S) and submit it online or by mail to the Immigration, Refugees and Citizenship Canada (IRCC) website.
4. **Pay the application fee**: Pay the required application fee, which is currently CAD 65.
5. *

llama_perf_context_print:        load time =    1449.26 ms
llama_perf_context_print: prompt eval time =    1160.51 ms /    10 tokens (  116.05 ms per token,     8.62 tokens per second)
llama_perf_context_print:        eval time =   32229.49 ms /   199 runs   (  161.96 ms per token,     6.17 tokens per second)
llama_perf_context_print:       total time =   33635.46 ms /   209 tokens
Llama.generate: 1 prefix-match hit, remaining 146 prompt tokens to eval


...

llama_perf_context_print:        load time =    1449.26 ms
llama_perf_context_print: prompt eval time =   20793.22 ms /   171 tokens (  121.60 ms per token,     8.22 tokens per second)
llama_perf_context_print:        eval time =    3942.81 ms /    25 runs   (  157.71 ms per token,     6.34 tokens per second)
llama_perf_context_print:       total time =   24764.63 ms /   196 tokens
Llama.generate: 1 prefix-match hit, remaining 12 prompt tokens to eval



Test Query 4: Do I need a new study permit if I change schools?
Response Before RAG:
 ?
If you're a Canadian student, you're likely familiar with the process of obtaining a study permit. However, if you're wondering if you need a new study permit if you change schools, the answer is a bit more complicated.

**General Rule:**
If you're a Canadian citizen or permanent resident, you'll need to apply for a new study permit if you're changing schools. This is because your student status is tied to your current school or university.

**Exceptions:**
However, there are some exceptions to this rule:

1. **You're a Canadian citizen or permanent resident who has been enrolled in a Canadian post-secondary institution for at least six months.** In this case, you won't need a new study permit, as your student status is tied to your current school.
2. **You're a Canadian citizen or permanent resident who has been enrolled in a Canadian post-secondary institution for less than six months.** In this 

llama_perf_context_print:        load time =    1449.26 ms
llama_perf_context_print: prompt eval time =    1390.90 ms /    12 tokens (  115.91 ms per token,     8.63 tokens per second)
llama_perf_context_print:        eval time =   32250.92 ms /   199 runs   (  162.06 ms per token,     6.17 tokens per second)
llama_perf_context_print:       total time =   33888.59 ms /   211 tokens
Llama.generate: 1 prefix-match hit, remaining 363 prompt tokens to eval


Query Results: {'ids': [['58bc43eb-269d-4b7e-8cd3-7f0eb6e50019', '543dce98-6fdc-4f93-98b1-3a3cc598d118', '7dc713d8-fa34-4beb-b5fb-2adad314edd4']], 'embeddings': None, 'documents': [['How long will it take to process my study permit application?("https://ircc.canada.ca/english/helpcentre/answer.asp?qnum=486&top=15") Processing times vary. It depends on the type of application you submit, and where it is processed. IRCC lists\xa0processing times("https://www.canada.ca/en/immigration-refugees-citizenship/services/application/check-processing-times.html")\xa0for applications submitted inside and outside Canada. We update this section regularly, so visit often to get the latest details.', 'How long will it take to process my study permit application?("https://ircc.canada.ca/english/helpcentre/answer.asp?qnum=486&top=15") Processing times vary. It depends on the type of application you submit, and where it is processed. IRCC lists\xa0processing times("https://www.canada.ca/en/immigration-ref

KeyboardInterrupt: 